In [66]:
#implementation of the NTRU encryption scheme
#see pp. \S 7.10 and pp. 490-494 Hoffstein, Pipher, Silverman, An Introduction to Mathematical Cryptography
#follow example 7.53 in [HPS]

In [67]:
#define the parameters
N=7; p=3; q=41; d=2
assert N.is_prime()
assert p.is_prime()
assert gcd(N,q) == 1 
assert gcd(p,q) == 1
assert q > (6*d+1)*p

In [68]:
#create the polynomial ring \Z[x]/(x^N-1)
P.<x> = PolynomialRing(ZZ)
R = P.quotient(x^N-1); R

Univariate Quotient Polynomial Ring in xbar over Integer Ring with modulus x^7 - 1

In [69]:
#a ternary polynomial in R = \Z[x]/(x^N-1) has d1 coeffs = +1, d2 coeffs = -1
def is_ternary(a,d1,d2):
    assert a in R
    return list(a).count(+1) == d1 and list(a).count(-1) == d2

In [70]:
#create the quotient polynomial ring \Z_p[x]/(x^N-1)
P_p.<x> = PolynomialRing(GF(p))
R_p = P_p.quotient(x^N - 1); R_p

Univariate Quotient Polynomial Ring in xbar over Finite Field of size 3 with modulus x^7 + 2

In [71]:
#create the quotient polynomial ring \Z_q[x]/(x^N-1)
P_q.<x> = PolynomialRing(GF(q))
R_q = P_q.quotient(x^N - 1); R_q

Univariate Quotient Polynomial Ring in xbar over Finite Field of size 41 with modulus x^7 + 40

In [72]:
homomorphic_encryption = True

In [73]:
#define the polynomial f(x). f(x) \in \mathcal{T}(d+1,d), i.e. is a ternary polynomial
#so has d+1 coeffs = 1, d coeffs = -1, all other coeffs = 0
if homomorphic_encryption:
    f = R([1,p,2*p,p,p,p,p])
    assert R_p(f) == 1
else:
    f = R([-1,0,1,1,-1,0,1])
    assert is_ternary(f,d+1,d)
print(f)
print(R_p(f))
print(R_q(f))

3*xbar^6 + 3*xbar^5 + 3*xbar^4 + 3*xbar^3 + 6*xbar^2 + 3*xbar + 1
1
3*xbar^6 + 3*xbar^5 + 3*xbar^4 + 3*xbar^3 + 6*xbar^2 + 3*xbar + 1


In [74]:
#define the polynomial g(x). g(x) \in \mathcal{T}(d,d), i.e. is a ternary polynomial
#so has d coeffs = 1, d coeffs = -1, all other coeffs = 0
if homomorphic_encryption:
    g = R([0,p,p,p,p,p,p])
    assert R_p(g) == 0
else:
    g = R([0,-1,-1,0,1,0,1])
    assert is_ternary(g,d,d)
print(g)
print(R_p(g))
print(R_q(g))

3*xbar^6 + 3*xbar^5 + 3*xbar^4 + 3*xbar^3 + 3*xbar^2 + 3*xbar
0
3*xbar^6 + 3*xbar^5 + 3*xbar^4 + 3*xbar^3 + 3*xbar^2 + 3*xbar


In [75]:
#compute F_p(x) = f(x)^{-1} (mod p)
F_p = R_p(f)^(-1); print(F_p)
assert F_p * R_p(f) == 1

1


In [76]:
#compute F_q(x) = f(x)^{-1} (mod q)
F_q = R_q(f)^(-1); print(F_q)
assert F_q * R_q(f) == 1

22*xbar^6 + 38*xbar^5 + 14*xbar^4 + 11*xbar^3 + 36*xbar^2 + 34*xbar + 37


In [77]:
#define the private key used to decrypt messages
def private_key(f):
    return (f,R_q(f)^(-1))

In [78]:
#define the public key using f(x) (mod p), g(x) (mod p)
def public_key(f,g,q):
    F_q = R_q(f)^(-1)
    return F_q * R_q(g)

In [79]:
print(public_key(f,g,q))

18*xbar^6 + 11*xbar^5 + xbar^4 + 10*xbar^3 + 17*xbar^2 + 23*xbar + 14


In [80]:
#define a plaintext message m(x) with coefficients satisfying -p/2 < m_i <= p/2
#i.e. m is the center lift of a polynomial in R_p
def plaintext(message,p,N):
    assert all([-p/2 < message[i] <= +p/2 for i in range(N)])
    return R(message)

In [81]:
#choose a random element of \mathcal{T}(d,d)
def rand_r(N,d):
    coeffs = (N-2*d)*[0] + d*[+1] + d*[-1]
    sigma = Permutations(N).random_element()
    r = R([coeffs[sigma[i]-1] for i in range(N)])
    assert is_ternary(r,d,d)
    return r

In [82]:
#compute the ciphertext e(x)
def ciphertext(message,f,g,r,p,q,N,d):
    h = public_key(f,g,q)
    m = plaintext(message,p,N)
    e = R_q(p*h*r + m)
    return e

In [83]:
#begin decryption process
def a(f,e):
    return R_q(f*e)

In [84]:
#define the center lift of a from R_q to an element of R = \Z[x]/(x^N-1)
#map coefficients from F_q to Z/qZ (the additive group) and center lift them to \Z
def center_lift(a,modulus):
    return R([Zmod(modulus)(coeff).lift_centered() for coeff in list(a)])

In [85]:
#compute b(x) = F_p(x) * a(x) (mod p) by first center lifting a(x) to R
def b(f,a,q):
    F_p = R_p(f)^(-1)
    return R_p(F_p * center_lift(a,q))

In [86]:
#define the final center_lift for decryption where e = ciphertext, f = secret key, p = prime, q = modulus
def decrypt(e,f,p,q):
    return center_lift(b(f,a(f,e),q),p)

In [87]:
#EXAMPLES

In [88]:
m = plaintext([1,-1,1,1,0,-1,0],p,N); m

-xbar^5 + xbar^3 + xbar^2 - xbar + 1

In [89]:
r = R([-1,1,0,0,0,-1,1]); r

xbar^6 - xbar^5 + xbar - 1

In [90]:
if homomorphic_encryption:
    assert all([abs(coeff) < q/2 for coeff in list(g*r+f*m)])

In [91]:
e = ciphertext(m,f,g,r,p,q,N,d); e

34*xbar^6 + 22*xbar^5 + 6*xbar^4 + 33*xbar^3 + 5*xbar^2 + 34*xbar + 31

In [92]:
a(f,e)

35*xbar^6 + 17*xbar^5 + 6*xbar^4 + 39*xbar^3 + 4*xbar^2 + 37*xbar + 7

In [93]:
center_lift(a(f,e),q)

-6*xbar^6 + 17*xbar^5 + 6*xbar^4 - 2*xbar^3 + 4*xbar^2 - 4*xbar + 7

In [94]:
b(f,a(f,e),q)

2*xbar^5 + xbar^3 + xbar^2 + 2*xbar + 1

In [95]:
decrypt(e,f,p,q)

-xbar^5 + xbar^3 + xbar^2 - xbar + 1